# Machine Learning Programming Exercise 6: Spam Classification with SVMs

In [1]:
#import package(s)
import numpy as np
import scipy.io as sio 
from sklearn import svm 
import regex as re
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

## 2 Spam Classification
In the next half of the exercise, support vector machines (SVM) are used to build a spam classiﬁer.

A classifier is trained to classify whether a given email, $x$, is spam ($y = 1$) or non-spam ($y = 0$). Each email needs to be converted into a feature vector $x \in \mathbb{R}^n$. The following parts of the exercise is a walk through of how such a feature vector can be constructed from an email. 

The dataset included for this exercise is based on a a subset of the [SpamAssassin Public Corpus](http://spamassassin.apache.org/old/publiccorpus/). For the purpose of this exercise, only the body of the email is used (excluding the email headers).

### 2.1 Preprocessing Emails
Before starting on a machine learning task, it is usually insightful to take a look at examples from the dataset.

Image Source: ex6.pdf from Andrew Ng's Coursera course in Machine Learning
![sampemail.png](sampemail.png)

Figure 8 shows a sample email that contains a URL, an email address (at the end), numbers, and dollar amounts. While many emails would contain similar types of entities (e.g., numbers, other URLs, or other email addresses), the speciﬁc entities (e.g., the speciﬁc URL or speciﬁc dollar amount) will be diﬀerent in almost every email. Therefore, one method often employed in processing emails is to “normalize” these values, so that all URLs are treated the same, all numbers are treated the same, etc. For example, we each URL in the email could be replace with the unique string "httpaddr" to indicate that a URL was present.

This has the eﬀect of letting the spam classiﬁer make a classiﬁcation decision based on whether any URL was present, rather than whether a speciﬁc URL was present. This typically improves the performance of a spam classiﬁer, since spammers often randomize the URLs, and thus the odds of seeing any particular URL again in a new piece of spam is very small.

In the function processEmail, the following email preprocessing and normalization steps are implemented:
* **Lower-casing**: The entire email is converted into lower case, so that captialization is ignored (e.g., IndIcaTE is treated the same as Indicate).
* **Stripping HTML**: All HTML tags are removed from the emails. Many emails often come with HTML formatting; we remove all the HTML tags, so that only the content remains.
* **Normalizing URLs**: All URLs are replaced with the text "httpaddr".
* **Normalizing Email Addresses**: All email addresses are replaced with the text "emailaddr".
* **Normalizing Numbers**: All numbers are replaced with the text "number".
* **Normalizing Dollars**: All dollar signs ($\$$) are replaced with the text "dollar".
* **Word Stemming**: Words are reduced to their stemmed form. For example, "discount", "discounts", "discounted" and "discounting" are all replaced with "discount". Sometimes, the Stemmer actually strips oﬀ additional characters from the end, so "include", "includes", "included", and "including" are all replaced with "includ".
* **Removal of non-words**: Non-words and punctuation have been removed. All white spaces (tabs, newlines, spaces) have all been trimmed to a single space character.

The result of these preprocessing steps is shown in Figure 9. While preprocessing has left word fragments and non-words, this form turns out to be much easier to work with for performing feature extraction.

Image Source: ex6.pdf from Andrew Ng's Coursera course in Machine Learning
![preproemail.png](preproemail.png)

#### 2.1.1 Vocabulary List
After preprocessing the emails, there is a list of words (e.g., Figure 9) for each email. The next step is to choose which words to use in the classiﬁer and which to leave out.

For this exercise, only the most frequently occuring words are chosen as the set of words considered (the vocabulary list). Since words that occur rarely in the training set are only in a few emails, they might cause the model to overﬁt the training set. The complete vocabulary list is in the ﬁle vocab.txt and also shown in Figure 10. The vocabulary list was selected by choosing all words which occur at least a 100 times in the spam corpus, resulting in a list of 1899 words. In practice, a vocabulary list with about 10,000 to 50,000 words is often used. 

Given the vocabulary list, each word can now be mapped in the preprocessed emails (e.g., Figure 9) into a list of word indices that contains the index of the word in the vocabulary list. Figure 11 shows the mapping for the sample email. Speciﬁcally, in the sample email, the word "anyone" was ﬁrst normalized to "anyon" and then mapped onto the index 86 in the vocabulary list.

Image Source: ex6.pdf from Andrew Ng's Coursera course in Machine Learning
![vocabind.png](vocabind.png)

Complete the code in processEmail to perform this mapping. In the code, a string str is a single word from the processed email. Look up the word in the vocabulary list vocabList and ﬁnd if the word exists in the vocabulary list. If the word exists, add the index of the word into the word indices variable. If the word does not exist, and is therefore not in the vocabulary, skip the word.

<div class="alert alert-block alert-info">
<b>Note:</b> 
    By using a stemming software that basically looks at the first few letters of a word, more of less, it can help, but it can also hurt. For example, the software may mistake the words universe and university as being the same thing because these two words start off with the same letters.
</div>

In [2]:
#define function for exercise(s)
def getVocabList():
#GETVOCABLIST reads the fixed vocabulary list in vocab.txt and returns a
#cell array of the words
#   vocabList = GETVOCABLIST() reads the fixed vocabulary list in vocab.txt 
#   and returns a cell array of the words in vocabList.

    #Read the fixed vocabulary list
    fid = open('vocab.txt')

    #Store all dictionary words in cell array vocab{}
    #n = 1899  # Total number of words in the dictionary
    
    #For ease of implementation, we use a struct to map the strings => integers
    #In practice, you'll want to use some form of hashmap
    vocabList = []
    for i in fid:
        index, word = i.split()
        vocabList.append(word)
    
    fid.close()
    return vocabList

def processEmail(email_contents):
#PROCESSEMAIL preprocesses a the body of an email and
#returns a list of word_indices 
#   word_indices = PROCESSEMAIL(email_contents) preprocesses 
#   the body of an email and returns a list of indices of the 
#   words contained in the email. 

    #Load Vocabulary
    vocabList = getVocabList()

    #Init return value
    word_indices = []

    # Preprocess Email 
    
    #Find the Headers ( \n\n and remove )
    #Uncomment the following lines if you are working with raw emails with the
    #full headers

    #hdrstart = strfind(email_contents, ([char(10) char(10)]));
    #email_contents = email_contents(hdrstart(1):end);
    
    #Lower case
    email_contents = [line.lower() for line in email_contents]
    
    #Strip all HTML
    #Looks for any expression that starts with < and ends with > and replace
    #and does not have any < or > in the tag it with a space
    email_contents = [re.sub('^[<]|[<.*>]|[>]$', '', line) for line in email_contents]
    
    #Handle Numbers
    #Look for one or more characters between 0-9
    email_contents = [re.sub('\d+', 'number', line) for line in email_contents]
    
    #Handle URLS
    #Look for strings starting with http:// or https://
    email_contents = [re.sub('(http|https)://+[^\s]+.*?', 'httpaddr', line) for line in email_contents]
    
    #Handle Email Addresses
    #Look for strings with @ in the middle
    email_contents = [re.sub('[^\s]+@[^\s]+', 'emailaddr', line) for line in email_contents]

    #Handle $ sign
    email_contents = [re.sub('[$]+', 'dollar', line) for line in email_contents]
    
    #========================== Tokenize Email ===========================

    #Output the email to screen as well
    print('\n==== Processed Email ====\n\n')

    #Process file
    
    #Get rid of any punctuation and leading and ending spaces 
    email_contents = [re.sub(r'[^\w\s]','',line) for line in email_contents]
    email_contents = [line.strip() for line in email_contents]
    
    #Remove any non alphanumeric characters
    #email_contents = [re.sub(r'\W+','',line) for line in email_contents]
    
    #Tokenize (split into an array of words)
    email_contents = word_tokenize(' '.join(email_contents))
    
    #Stem the word 
    #(the porterStemmer sometimes has issues, so we use a try catch block)
    ps = PorterStemmer()
    stem_word = []
    for w in email_contents:
        stem_word.append(ps.stem(w))
    
    #Look up the word in the dictionary and add to word_indices if found
    #add one since Python starts its index at zero instead of one
    for i in range(len(stem_word)):
        if stem_word[i] in vocabList:
            word_indices.append((vocabList.index(stem_word[i]))+1)
        
    return word_indices
    
# ==================== Part 1: Email Preprocessing ====================
#  To use an SVM to classify emails into Spam v.s. Non-Spam, you first need
#  to convert each email into a vector of features. In this part, you will
#  implement the preprocessing steps for each email. You should
#  complete the code in processEmail.m to produce a word indices vector
#  for a given email.

print('\nPreprocessing sample email (emailSample1.txt)\n')

#Extract Features
file_contents = open('emailSample1.txt')
word_indices  = processEmail(file_contents)

#Print Stats
print('Word Indices: \n')
print(word_indices)
print('\n\n')


Preprocessing sample email (emailSample1.txt)


==== Processed Email ====


Word Indices: 

[86, 916, 794, 1077, 883, 370, 1699, 790, 1822, 1831, 883, 431, 1171, 794, 1002, 1895, 592, 1676, 238, 162, 89, 688, 945, 1663, 1120, 1062, 1699, 375, 1162, 479, 1893, 1510, 799, 1182, 1237, 810, 1895, 1440, 1547, 181, 1699, 1758, 1896, 688, 1676, 992, 961, 1477, 71, 530, 1699, 531]





<div class="alert alert-block alert-info">
<b>Implementation Note:</b> 
    To find the index of a matched string in vocabList use vocabList.index(str).
    Speciﬁcally, to get the word at index $i$, you can use vocabList[i].
    Use len(vocabList) to get the number of words in the vocabulary.
</div>

### 2.2 Extracting Features from Emails
Implement the feature extraction that converts each email into a vector in $\mathbb{R}^n$. For this exercise, use $n = \#$ words in vocabulary list. Specifically, the feature $x_i \in \{0,1\}$ for an email corresponds to whether the $i$-th word in the dictionary occurs in the email. That is, $x_i = 1$ if the $i$-th word is in the email and $x_i = 0$ if the $i$-th word is not present in the email. 

Thus, for a typical email, this feature would look like:
$x = \begin{bmatrix}
       0 \\
       \vdots \\
       1 \\
       0 \\
       \vdots \\
       1 \\ 
       0 \\
       \vdots \\
       0
       \end{bmatrix} \in \mathbb{R}^n.$
       
Implement the function emailFeatures on the email sample. The feature vector should had a length $1899$ and $45$ non-zero entries.

In [3]:
#define function for exercise(s)
def emailFeatures(word_indices):
#EMAILFEATURES takes in a word_indices vector and produces a feature vector
#from the word indices
#   x = EMAILFEATURES(word_indices) takes in a word_indices vector and 
#   produces a feature vector from the word indices.     
    
    #Total number of words in the dictionary
    vocabList = getVocabList()
    n = len(vocabList)

    #You need to return the following variables correctly.
    feat_vec = np.zeros((n, 1))
    
    for index in range(n):
        if index in word_indices:
            feat_vec[index-1]=1
    return feat_vec

# ==================== Part 2: Feature Extraction ====================
#  Now, you will convert each email into a vector of features in R^n. 
#  You should complete the code in emailFeatures.m to produce a feature
#  vector for a given email.

print('\nExtracting features from sample email (emailSample1.txt)\n')

#Extract Features
features = emailFeatures(word_indices)

#Print Stats
print('Length of feature vector:', len(features))
print('Number of non-zero entries:', np.sum(features > 0))


Extracting features from sample email (emailSample1.txt)

Length of feature vector: 1899
Number of non-zero entries: 44


### 2.3 Training SVM for Spam Classiﬁcation
After completing the feature extraction functions, the next step to train a SVM classfier with a preprocessed training dataset. spamTrain.mat contains $4000$ training examples of spam and non-spam email, while spamTest.mat contains $1000$ test examples. Each original email was processed using the processEmail and emailFeatures functions and converted into a vector $x^{(i)} \in \mathbb{R}^{1899}$.

After loading the dataset, the SVM is trained to classify between spam $(y = 1)$ and non-spam $(y = 0)$ emails. Once the training completes, the classiﬁer should get a training accuracy of about $99.8\%$ and a test accuracy of about $98.5\%.$

<div class="alert alert-block alert-info">
<b>Note:</b> 
    It is very important to get error results as a single, numerical value otherwise it is difficult to assess your algorithm’s performance. For example if we use stemming, which is the process of treating the same word with different forms (fail/failing/failed) as one word (fail), and get a $3\%$ error rate instead of $5\%$, then we should definitely add it to our model. However, if we try to distinguish between upper case and lower case letters and end up getting a $3.2\%$ error rate instead of $3\%$, then we should avoid using this new feature. Hence, we should try new things and get a numerical value for our error rate, then based on our result decide whether we want to keep the new feature or not. In addition, it is strongly recommended to do error analysis is on the cross validations rather than the test set. 
</div>

In [4]:
# =========== Part 3: Train Linear SVM for Spam Classification ========
#  In this section, you will train a linear classifier to determine if an
#  email is Spam or Not-Spam.

#Load the Spam Email dataset
#You will have X, y in your environment
train_data = sio.loadmat('spamTrain.mat')
X = train_data['X']
y = train_data['y']

print('\nTraining Linear SVM (Spam Classification)\n')
print('(this may take 1 to 2 minutes) ...\n')

C=0.1
model = (svm.LinearSVC(C=C, loss='hinge', tol=0.001, max_iter=1000)).fit(X,y.ravel())
p_train = model.predict(X)

print('Train Accuracy: ', (np.mean(np.equal(p_train, y.T)))*100)
print('Expected accuracy (approx): 99.8\n')


Training Linear SVM (Spam Classification)

(this may take 1 to 2 minutes) ...

Train Accuracy:  99.85000000000001
Expected accuracy (approx): 99.8



In [5]:
# =================== Part 4: Test Spam Classification ================
#  After training the classifier, we can evaluate it on a test set. We have
#  included a test set in spamTest.mat

#Load the test dataset
#You will have Xtest, ytest in your environment
test_data = sio.loadmat('spamTest.mat')
Xtest = test_data['Xtest']
ytest = test_data['ytest']

print('\nEvaluating the trained Linear SVM on a test set ...\n')
p_test = model.predict(Xtest)

print('Test Accuracy: ', (np.mean(np.equal(p_test, ytest.T)))*100)
print('Expected accuracy (approx): 98.5\n')


Evaluating the trained Linear SVM on a test set ...

Test Accuracy:  98.9
Expected accuracy (approx): 98.5



### 2.4 Top Predictors for Spam
To better understand how the spam classiﬁer works, the parameters are inspected to see which words the classifier thinks are the most predictive of spam. The next step is to find the parameters with the largest positive values in the classiﬁer and displays the corresponding words (Figure 12). Thus, if an email contains words such as "guarantee", "remove", "dollar", and "price" (the top predictors shown in Figure 12), it is likely to be classiﬁed as spam.

Image Source: ex6.pdf from Andrew Ng's Coursera course in Machine Learning
![topspam.png](topspam.png)

In [6]:
# ================= Part 5: Top Predictors of Spam ====================
#  Since the model we are training is a linear SVM, we can inspect the
#  weights learned by the model to understand better how it is determining
#  whether an email is spam or not. The following code finds the words with
#  the highest weights in the classifier. Informally, the classifier
#  'thinks' that these words are the most likely indicators of spam.

#Sort the weights and obtin the vocabulary list
modelw = (model.coef_)*np.dot(y.T,X)
weights = -np.sort(-modelw)
idx = np.argsort(-modelw)
vocabList = getVocabList()

print('\nTop predictors of spam: \n')
print('vocab word  weights\n')
for i in range(15):
    print(vocabList[idx[0][i]], weights[0][i])


Top predictors of spam: 

vocab word  weights

visit 87.19308359962946
click 86.65099398452745
guarante 74.28045564391603
remov 60.806342183571836
price 59.575379325485805
dollar 59.51109928647399
hour 54.86028766311198
most 51.233909660779545
credit 46.858530917173766
you 42.662159218649386
repli 41.86440208284136
contact 40.447059133356184
pleas 40.16580151991224
nbsp 40.04630836026987
pai 37.63121072291729


### 2.5 Optional (ungraded) exercise: Try your own emails
In the starter code, two email examples (emailSample1.txt and emailSample2.txt) and two spam examples (spamSample1.txt and spamSample2.txt) are included. The last part of the exercise involves running the spam classfier over the ﬁrst spam example and classifies it using the learned SVM. Try the other examples provided and see if the classiﬁer gets them right. You can also try your own emails by replacing the examples (plain text ﬁles) with your own emails.

In [7]:
# =================== Part 6: Try Your Own Emails =====================
#  Now that you've trained the spam classifier, you can use it on your own
#  emails! In the starter code, we have included spamSample1.txt,
#  spamSample2.txt, emailSample1.txt and emailSample2.txt as examples. 
#  The following code reads in one of these emails and then uses your 
#  learned SVM classifier to determine whether the email is Spam or 
#  Not Spam 

#Set the file to be read in (change this to spamSample2.txt,
#emailSample1.txt or emailSample2.txt to see different predictions on
#different emails types). Try your own emails as well!
filename = 'spamSample1.txt'

#Read and predict
file_contents = open(filename)
word_indices  = processEmail(file_contents)
x             = emailFeatures(word_indices)

p_spam = model.predict(x.T)

print('\nProcessed', filename)
print('Spam Classification: ', p_spam)
print('(1 indicates spam, 0 indicates not spam)\n\n')


==== Processed Email ====



Processed spamSample1.txt
Spam Classification:  [1]
(1 indicates spam, 0 indicates not spam)




In [8]:
#Set the file to be read in (change this to spamSample2.txt,
#emailSample1.txt or emailSample2.txt to see different predictions on
#different emails types). Try your own emails as well!
filename = 'spamSample2.txt'

#Read and predict
file_contents = open(filename)
word_indices  = processEmail(file_contents)
x             = emailFeatures(word_indices)

p_spam = model.predict(x.T)

print('\nProcessed', filename)
print('Spam Classification: ', p_spam)
print('(1 indicates spam, 0 indicates not spam)\n\n')


==== Processed Email ====



Processed spamSample2.txt
Spam Classification:  [1]
(1 indicates spam, 0 indicates not spam)




In [9]:
#Set the file to be read in (change this to spamSample2.txt,
#emailSample1.txt or emailSample2.txt to see different predictions on
#different emails types). Try your own emails as well!
filename = 'emailSample1.txt'

#Read and predict
file_contents = open(filename)
word_indices  = processEmail(file_contents)
x             = emailFeatures(word_indices)

p_spam = model.predict(x.T)

print('\nProcessed', filename)
print('Spam Classification: ', p_spam)
print('(1 indicates spam, 0 indicates not spam)\n\n')


==== Processed Email ====



Processed emailSample1.txt
Spam Classification:  [0]
(1 indicates spam, 0 indicates not spam)




In [10]:
#Set the file to be read in (change this to spamSample2.txt,
#emailSample1.txt or emailSample2.txt to see different predictions on
#different emails types). Try your own emails as well!
filename = 'emailSample2.txt'

#Read and predict
file_contents = open(filename)
word_indices  = processEmail(file_contents)
x             = emailFeatures(word_indices)

p_spam = model.predict(x.T)

print('\nProcessed', filename)
print('Spam Classification: ', p_spam)
print('(1 indicates spam, 0 indicates not spam)\n\n')


==== Processed Email ====



Processed emailSample2.txt
Spam Classification:  [0]
(1 indicates spam, 0 indicates not spam)




### 2.6 Optional (ungraded) exercise: Build your own dataset
In this exercise, a preprocessed training set and test set was provided. These datasets were created using the same functions (processEmail and emailFeatures). For this optional (ungraded) exercise, build your own dataset using the original emails from the [SpamAssassin Public Corpus](http://spamassassin.apache.org/old/publiccorpus/).

The task in this optional (ungraded) exercise is to download the original files from the public corpus and extract them. After extracting them, run the processEmail and emailFeatures functions on each email to extract a feature vector from each email to build a dataset $X, y$ of examples. Then randomly divide up the dataset into a training set, a cross validation set and a test set. Use processEmail to remove the email headers from the emails.

When building your own dataset, try building your own vocabulary list by selecting the high frequency word that occur in the dataset and adding any additional features that you think might be useful. Try to use a highly optimized SVM toolboxes as well.